# Now let's monitor the training/deploying process

In [1]:
import boto3
import ipywidgets as widgets
import time

from IPython.display import display

## Helper functions

In [2]:
def get_actions():
    actions = []
    executionId = None
    resp = codepipeline.get_pipeline_state( name=pipeline_name )
    for stage in resp['stageStates']:
        stageName = stage['stageName']
        stageStatus = None
        if stage.get('latestExecution') is not None:
            stageStatus = stage['latestExecution']['status']
            if executionId is None:
                executionId = stage['latestExecution']['pipelineExecutionId']
            elif stage['latestExecution']['pipelineExecutionId'] != executionId:
                stageStatus = 'Old'
        for action in stage['actionStates']:
            actionName = action['actionName']
            actionStatus = 'Old'
            if action.get('latestExecution') is not None and stageStatus != 'Old':
                actionStatus = action['latestExecution']['status']
            actions.append( {'stageName': stageName, 
                             'stageStatus': stageStatus, 
                             'actionName': actionName, 
                             'actionStatus': actionStatus})
    return actions

In [3]:
def get_approval_token():
    resp = codepipeline.get_pipeline_state( name=pipeline_name )
    token = None
    # Get the approve train status token
    for stageState in resp['stageStates']:
        if stageState['stageName'] == 'DeployApproval':
            for actionState in stageState['actionStates']:
                if actionState['actionName'] == 'ApproveDeploy':
                    if actionState.get('latestExecution') is None:
                        return None
                    latestExecution = actionState['latestExecution']
                    if latestExecution['status'] == 'InProgress':
                        token = latestExecution['token']
    return token

In [4]:
from sagemaker.serializers import CSVSerializer
csv_serializer = CSVSerializer()
def test_endpoint(endpoint_name, payload):
    resp = sm.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',
        Accept='text/csv',
        Body=csv_serializer.serialize(payload)
    )
    variant_name = resp['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
    return float(resp['Body'].read().decode('utf-8').strip()), variant_name

In [5]:
def approval(token, result):
    if token is None:
        return
    
    codepipeline.put_approval_result(
      pipelineName=pipeline_name,
      stageName='DeployApproval',
      actionName='ApproveDeploy',
      result=result,
      token=token
    )

In [6]:
def approve(b):
    result={
        'summary': 'This is a great model! Put into production.',
        'status': 'Approved'
    }
    approval(get_approval_token(), result) 
    button_box.close()
    start_monitoring()

In [7]:
def reject(b):
    result={
        'summary': 'This is a rubbish model. Discard it',
        'status': 'Rejected'
    }
    approval(get_approval_token(), result)
    button_box.close()
    start_monitoring()

In [8]:
def start_monitoring():
    global button_box
    
    running = True
    while running:
        steps_ok = 0
        for k,action in enumerate(get_actions()):
            if action['actionStatus'] == 'Failed':
                bar.bar_style='danger'
                label.value='Ops! Something went wrong Stage[{}] Action[{}]'.format(
                    action['stageName'], action['actionName'])
                running = False
                return

            elif action['actionStatus'] == 'InProgress':
                if get_approval_token() is not None:
                    display(button_box)
                    running = False
                break
            elif action['actionStatus'] == 'Old':
                break
            elif action['actionStatus'] == 'Succeeded':
                steps_ok += 1
        
        label.value = "Actions {}/{} - Current: Stage[{}] Action[{}]".format( 
                k+1,max_actions, action['stageName'], action['actionName'] )
        bar.value = steps_ok

        if steps_ok == max_actions:
            running = False
        else:    
            time.sleep(2)

## Job monitoring

In [9]:
codepipeline = boto3.client('codepipeline')
sm = boto3.client('sagemaker-runtime')

model_prefix='iris-model'
pipeline_name = 'iris-model-pipeline'
endpoint_name_mask='{}-%s'.format(model_prefix)

In [10]:
approve_btn = widgets.Button(description="Approve", button_style='success', icon='check')
reject_btn = widgets.Button(description="Reject", button_style='danger', icon='close')
approve_btn.on_click(approve)
reject_btn.on_click(reject)
button_box = widgets.HBox([approve_btn, reject_btn])
                
max_actions = len(get_actions())
label = widgets.Label(value="Loading...")
bar = widgets.IntProgress( value=0, min=0, max=max_actions, step=1, bar_style='info' )
info_box = widgets.VBox([label, bar])

display(info_box)
start_monitoring()

PipelineNotFoundException: An error occurred (PipelineNotFoundException) when calling the GetPipelineState operation: Account '686708870566' does not have a pipeline with name 'iris-model-pipeline'

## Now, if everything went fine, we can test our models

In [ ]:
%%time
payload = [4.6, 3.1, 1.5, 0.2]

print( "DSV")
print( "Classifier: %s, Variant Name: %s" % test_endpoint( endpoint_name_mask % ('development'), payload ) )

print( "\nPRD")
print( "Classifier: %s, Variant Name: %s" % test_endpoint( endpoint_name_mask % ('production'), payload ) )